In [2]:

def GSO(B,N):
    B_prime = Matrix(QQ,N)
    mu=Matrix(QQ,N)
    for i in range(n):
        B_prime[i] = B[i]
        mu[i,i]=1
        for j in range(i):
            mu[i,j] = B[i].inner_product(B_prime[j])/B_prime[j].norm()^2
            B_prime[i] -= mu[i,j]*B_prime[j]
    return B_prime,mu


#deep insertion後のGSO情報を効率的に更新
def GSOUpdate_DeepLLL(mu,norm_B,i,k,N):
    #output: Deep Insetion後のGSO係数行列とGSOベクトルの2乗ノルム
    P = vector(QQ,N)
    D = vector(QQ,N)
    P[k] = norm_B[k]
    D[k] = norm_B[k]
    for j in reversed(range(i,k)):
        P[j] = mu[k,j]*norm_B[j]
        #Dj = ||πj(bk)||^2
        D[j] = D[j+1] + mu[k,j]*P[j]
    
    S = vector(QQ,N)
    for j in reversed(range(i+1,k+1)):
        T = mu[k,j-1]/D[j]
        for l in reversed(range(k+1,N)):
            S[l] = S[l] + mu[l,j]*P[j]
            mu[l,j] = mu[l,j-1] - T*S[l]
            
        for l in reversed(range(j+1,k+1)):
            S[l] += mu[l-1,j]*P[j]
            mu[l,j] = mu[l-1,j-1] - T*S[l]
    T = 1/D[i]
    
    for l in reversed(range(k+1,N)):
        mu[l,i] = T*(S[l] + mu[l,i]*P[i])
    for l in reversed(range(i+2,k+1)):
        mu[l,i] = T*(S[l] +mu[l-1,i]*P[i])
    
    mu[i+1,i] = T*P[i]
    
    for j in range(i):
        ips = mu[k,j]
        for l in reversed(range(i+1,k+1)):
            mu[l,j] = mu[l-1,j]
        mu[i,j] = ips
        
    for j in reversed(range(i+1,k+1)):
        norm_B[j] = (D[j]*norm_B[j-1])/D[j-1]
    
    norm_B[i] = D[i]
    return mu,norm_B
        
        
    




def DeepLLL(B,delta,N):
    B_prime,mu = GSO(B,N)
    norm_B = vector(QQ,N)
    for p in range(N):
        norm_B[p] = B_prime[p].norm()^2
    k = 1
    
    while k <= N-1:
        #Size_reduce
        for j in reversed(range(k)):
            if abs(mu[k,j]) > 0.50:
                q = round(mu[k,j])
                B[k] -= q*B[j]
                for l in range(j+1):
                    mu[k,l] -= q*mu[j,l]
            #print("Size_reduce",k,j,B,mu)
        C = B[k].norm()^2
        i=0
        while i < k:
            #C = ||πi(bk)||^2を計算
            if C >= delta*norm_B[i]:
                C -= (mu[k,i]^2)*norm_B[i];i +=1;
            else:
                v = B[k]
                #Deep Insertion
                for j in reversed(range(i+1,k+1)):
                    B[j] = B[j-1]
                B[i] = v
                mu,norm_B = GSOUpdate_DeepLLL(mu,norm_B,i,k,N)
                #print("GSOUP",i,k,B,mu,norm_B)
                k = max(i,1)-1
        k+=1
    return B
                    
BB = Matrix([[84,3,34,17],[20,48,66,19],[69,14,63,78],[28,72,36,57]])
n=4
DeepLLL(BB,0.75,n)

        

[  8  24 -30  38]
[-43 -61  -7   4]
[-23 -13  59  23]
[ 41 -58  27  21]